In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os.path import join
from scipy.stats import ttest_rel
import warnings

warnings.filterwarnings("ignore")

In [ ]:
PROJ_DIR = "/Volumes/projects_herting/LABDOCS/Personnel/Katie/deltaABCD_clustering/"
DATA_DIR = "data/"
FIGS_DIR = "figures/"
OUTP_DIR = "output/"

In [ ]:
df = pd.read_pickle(join(PROJ_DIR, DATA_DIR, "data_qcd.pkl"))
# grap ppt ids for people who pass QC
ppts = df.filter(regex="rsfmri_c_.*change_score").dropna().index
#df = None
# we need ids for the vectorized upper triangle!
network_names = ['dt', 'ca', 'smh', 'dla', 'ad', 'smm', 'sa', 'fo', 'vs', 'cgc', 'vta', 'rspltp']

In [ ]:
#reading in the dataframes for each type of change score
sign_change = pd.read_pickle(join(PROJ_DIR, DATA_DIR, 'delta_rsFC-sign_changes.pkl'))
graph_df = pd.read_pickle(join(PROJ_DIR, OUTP_DIR, 'delta_rsFC-graph_measures-global.pkl'))
local_df = pd.read_pickle(join(PROJ_DIR, OUTP_DIR, 'delta_rsFC-graph_measures-local.pkl'))

In [ ]:
#reading in the dataframes for each fc descriptive stats
netwk_wise_desc = pd.read_pickle(join(PROJ_DIR, OUTP_DIR, 'netwk_wise_desc.pkl'))
within_netwk_desc = pd.read_pickle(join(PROJ_DIR, OUTP_DIR, 'within_netwk_desc.pkl'))
between_netk_desc = pd.read_pickle(join(PROJ_DIR, OUTP_DIR, 'between_netk_desc.pkl'))

In [ ]:
#reading in the melted multi-indexed dataframes for each fc descriptive stats
melt_ntwk_wise = pd.read_pickle(join(PROJ_DIR, OUTP_DIR, 'melt_ntwk_wise.pkl'))p
melt_wthn_ntwk = pd.read_pickle(join(PROJ_DIR, OUTP_DIR, 'melt_wthn_ntwk.pkl'))
melt_btwn_ntwk = pd.read_pickle(join(PROJ_DIR, OUTP_DIR, 'melt_btwn_ntwk.pkl'))

In [ ]:
sex = df[["sex.baseline_year_1_arm_1"]]
sex=sex.dropna()
print(sex)

In [ ]:
# I'm just interested in ppts who pass the quality control process
quality_ppts = list(set(ppts) & set(graph_df.index))

graph_df = graph_df.loc[quality_ppts]
local_df = local_df.loc[quality_ppts]
sex_df = sex.loc[quality_ppts]



In [ ]:
sex_index = sex_df.rename_axis('src_subject_id').reset_index()
sex_index.head()
#sex_index.to_csv(join(PROJ_DIR, OUTP_DIR,'sex_index.csv'))


In [ ]:
# creating the sex df for baseline 

base_sex = pd.read_csv(join(PROJ_DIR, OUTP_DIR, 'sex_index.csv'), index_col=0)
base_sex.set_index('src_subject_id', inplace=True)
base_sex.columns = pd.MultiIndex.from_product([['baseline_year_1_arm_1'], ['sex']])
base_sex



In [ ]:
# creating the sex df for y2fu

y2fu_sex = pd.read_csv(join(PROJ_DIR, OUTP_DIR, 'sex_index.csv'), index_col=0)
y2fu_sex.set_index('src_subject_id', inplace=True)
y2fu_sex.columns = pd.MultiIndex.from_product([['2_year_follow_up_y_arm_1'], ['sex']])
y2fu_sex



In [ ]:
# CONCAT SEX AND GRAPH_DF, THEN MELT

graph_df_sex = pd.concat([graph_df, base_sex, y2fu_sex], axis=1)
graph_df_sex[('baseline_year_1_arm_1','subject_id')] = graph_df_sex.index
graph_sex_melt = graph_df_sex.melt(id_vars=[('baseline_year_1_arm_1','subject_id'),
                                           ('baseline_year_1_arm_1','sex'),
                                           ('2_year_follow_up_y_arm_1','sex')])
cols=graph_sex_melt.columns
#cols

# reaname the columns

graph_sex_melt.columns = [('subject_id'), ('sex'), ('na'), ('variable_0'), ('variable_1'), ('value')]
graph_sex_melt



In [ ]:
local_df.dropna()



In [ ]:
# setting the variable ids

local_df['subject_id'] = local_df.index.get_level_values(0)
long_local = local_df.melt(id_vars=['timepoint', 'subject_id'])
long_local



In [ ]:
# save out long_local df for Katie

long_local.to_csv(join(PROJ_DIR, OUTP_DIR,'long_local.csv'))



In [ ]:
#GLOBAL spaghetti plot

sns.set_theme(style="ticks")
palette = sns.color_palette('muted')
ax= sns.relplot(graph_df_melt, 
            kind = "line",
            x="variable_0", 
            y="value", units="src_subject_id", hue="variable_1", col='variable_1', palette=palette,
            alpha=0.05, height=5, aspect=.75,facet_kws=dict(sharex=False, sharey=False), estimator=None,
            )



In [ ]:
#GLOBAL SEX spaghetti plot

sns.set_theme(style="ticks")
palette = sns.color_palette('Set2')
ax= sns.relplot(graph_sex_melt, 
            kind = "line",
            x="variable_0", 
            y="value", units="subject_id", hue="sex", col='variable_1', palette=palette,
            alpha=0.07, height=5, aspect=.75,facet_kws=dict(sharex=False, sharey=False), estimator=None,
            )


In [ ]:
#LOCAL spaghetti plot

sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
palette = sns.color_palette("rocket_r")

# Plot the lines on two facets
sns.relplot(long_local, 
            kind = "line",
            x="timepoint", 
            y="value", col='variable_0' , hue="variable_1", row="variable_1", units="subject_id",
            alpha=0.05, height=5, aspect=.75,facet_kws=dict(sharex=False, sharey=False), estimator=None,
            )

In [ ]:
# NETWORK WISE CONNECTIVITY DESCRIPTIVES BOXPLOT

sns.set_theme(style="ticks")

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(7, 10))
ax.set_xscale("symlog")


# Plot the data
sns.boxplot(
    melt_ntwk_wise, x="value", y="variable_1", hue="variable_0",
    palette="tab10", whis=50, fliersize=0)



In [ ]:
# WITHIN NETWORK CONNECTIVITY DESCRIPTIVES BOXPLOT

sns.set_theme(style="ticks")

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(7, 10))
ax.set_xscale("symlog")


# Plot the data
sns.boxplot(
    melt_wthn_ntwk, x="value", y="variable_1", hue="variable_0",
    palette="tab10", whis=50, fliersize=0)


In [ ]:
# BETWEEN NETWORK CONNECTIVITY DESCRIPTIVES BOXPLOT

sns.set_theme(style="ticks")

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(7, 10))
ax.set_xscale("symlog")


# Plot the data
sns.boxplot(
    melt_btwn_ntwk, x="value", y="variable_1", hue="variable_0",
    palette="tab10", whis=50, fliersize=0)
